## Importing Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## Loading Data

In [ ]:
dataset = pd.read_csv('Loan_Application.csv')

In [ ]:
dataset.head()

,CustomerNumber,CreditScore,City,Gender,Age,Home_Living_Since,Avg_Acc_Bal,Assets,First Loan,Co-applicant,Mon_Sal,Loan
0,1,619,Bangalore,Female,41,2,0.00,1,1,1,50674.440,1
1,2,608,Hyderabad,Female,40,1,41903.93,1,0,1,56271.290,0
2,3,502,Bangalore,Female,41,8,79830.40,3,1,0,56965.785,1
3,4,699,Bangalore,Female,38,1,0.00,2,0,0,46913.315,0
4,5,850,Hyderabad,Female,42,2,62755.41,1,1,1,39542.050,0


## Data Pre-processing

### Dummy Variables



In [ ]:
dataset_dummy = pd.get_dummies(dataset, drop_first=True)

In [ ]:
dataset_dummy.head()

,CustomerNumber,CreditScore,Age,Home_Living_Since,Avg_Acc_Bal,Assets,First Loan,Co-applicant,Mon_Sal,Loan,City_Chennai,City_Hyderabad,Gender_Male
0,1,619,41,2,0.00,1,1,1,50674.440,1,0,0,0
1,2,608,40,1,41903.93,1,0,1,56271.290,0,0,1,0
2,3,502,41,8,79830.40,3,1,0,56965.785,1,0,0,0
3,4,699,38,1,0.00,2,0,0,46913.315,0,0,0,0
4,5,850,42,2,62755.41,1,1,1,39542.050,0,0,1,0


In [ ]:
dataset_dummy.columns

Index(['CustomerNumber', 'CreditScore', 'Age', 'Home_Living_Since',
       'Avg_Acc_Bal', 'Assets', 'First Loan', 'Co-applicant', 'Mon_Sal',
       'Loan', 'City_Chennai', 'City_Hyderabad', 'Gender_Male'],
      dtype='object')

In [ ]:
X = dataset_dummy.loc[:, ['CreditScore', 'Age', 'Home_Living_Since',
       'Avg_Acc_Bal', 'Assets', 'First Loan', 'Co-applicant', 'Mon_Sal',       
       'City_Chennai', 'City_Hyderabad', 'Gender_Male']].values

In [ ]:
y = dataset_dummy.loc[:, ['Loan']].values

### Training-Testing Splitting

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Normalization - Feature Standardization

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Model Building - ANN


In [ ]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

### NN Architecture

In [ ]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, activation = 'relu', input_dim = 11))

In [ ]:
# Adding regularization

#classifier.add(Dropout(rate = 0.1))

Dropout parameter of 0.1 means 10% of the neurons would be disabled at each iteration.
If required, increase by 0.1 to improve the accuracy until 0.5.
So, that means p = 1, means no neurons and that is under-fitting.
In general, don't go over 0.5.

In [ ]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, activation = 'relu'))

In [ ]:
# Adding regularization
# classifier.add(Dropout(rate = 0.1))

In [ ]:
# Adding the output layer
classifier.add(Dense(units = 1, activation = 'sigmoid'))

### Model Compilation

In [ ]:
# Compiling the ANN
classifier.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Model Fitting

In [ ]:
# Fitting the ANN to the Training set

classifier.fit(X_train, y_train, batch_size = 10, epochs = 5, validation_data=(X_test, y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 4s 503us/step - loss: 0.3469 - acc: 0.8604 - val_loss: 0.3440 - val_acc: 0.8605
Epoch 2/5
8000/8000 [==============================] - 4s 489us/step - loss: 0.3459 - acc: 0.8606 - val_loss: 0.3466 - val_acc: 0.8570
Epoch 3/5
8000/8000 [==============================] - 4s 483us/step - loss: 0.3454 - acc: 0.8614 - val_loss: 0.3429 - val_acc: 0.8640
Epoch 4/5
8000/8000 [==============================] - 4s 488us/step - loss: 0.3445 - acc: 0.8611 - val_loss: 0.3456 - val_acc: 0.8575
Epoch 5/5
8000/8000 [==============================] - 4s 479us/step - loss: 0.3435 - acc: 0.8619 - val_loss: 0.3414 - val_acc: 0.8635


## Model Evaluation

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1543   52]
 [ 291  114]]


## Early Stopping and Model Check Point

In [ ]:
early_stoppping = EarlyStopping(monitor='val_loss', verbose=1, patience=10)
best_model = ModelCheckpoint('best_model.h5', monitor='val_acc', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [ ]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 5, validation_data=(X_test, y_test), callbacks=[early_stoppping, best_model, reduce_lr ])

Epoch 1/5
800/800 [==============================] - 2s 2ms/step - loss: 0.4044 - accuracy: 0.8232 - val_loss: 0.3943 - val_accuracy: 0.8285
Epoch 2/5
800/800 [==============================] - 2s 2ms/step - loss: 0.3994 - accuracy: 0.8246 - val_loss: 0.3917 - val_accuracy: 0.8275
Epoch 3/5
800/800 [==============================] - 2s 2ms/step - loss: 0.3948 - accuracy: 0.8246 - val_loss: 0.3857 - val_accuracy: 0.8330
Epoch 4/5
800/800 [==============================] - 2s 2ms/step - loss: 0.3913 - accuracy: 0.8263 - val_loss: 0.3839 - val_accuracy: 0.8260
Epoch 5/5
800/800 [==============================] - 2s 2ms/step - loss: 0.3877 - accuracy: 0.8256 - val_loss: 0.3813 - val_accuracy: 0.8270


## Grid Search CV

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
 
    classifier.add(Dense(units = 15, activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 10, activation = 'relu'))
    #classifier.add(Dropout(p = 0.1))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
    #classifier.add(Dropout(p = 0.1))    
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 
    return classifier
    
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [100, 300],
              #'epochs': [100, 500],
              'epochs': [2, 3],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                            cv=2) #10 

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/2
40/40 [==============================] - 0s 2ms/step - loss: 0.5394 - accuracy: 0.7915
Epoch 2/2
40/40 [==============================] - 0s 2ms/step - loss: 0.4974 - accuracy: 0.7983
Epoch 1/2
40/40 [==============================] - 0s 2ms/step - loss: 0.8121 - accuracy: 0.3433
Epoch 2/2
40/40 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 0.7320
Epoch 1/2
40/40 [==============================] - 0s 2ms/step - loss: 0.6856 - accuracy: 0.5960
Epoch 2/2
40/40 [==============================] - 0s 2ms/step - loss: 0.5557 - accuracy: 0.7567
Epoch 1/2
40/40 [==============================] - 0s 2ms/step - loss: 0.7653 - accuracy: 0.4575
Epoch 2/2
40/40 [==============================] - 0s 2ms/step - loss: 0.6004 - accuracy: 0.7040
Epoch 1/3
40/40 [==============================] - 0s 2ms/step - loss: 0.7936 - accuracy: 0.3755
Epoch 2/3
40/40 [==============================] - 0s 2ms/step - loss: 0.6256 - accuracy: 0.6895
Epoch 3/3
40/40 [=============

In [ ]:
best_parameters = grid_search.best_params_  
best_accuracy = grid_search.best_score_ 

print('best_parameters: ', best_parameters)
print('best accuracy: ', best_accuracy)

best_parameters:  {'batch_size': 100, 'epochs': 3, 'optimizer': 'rmsprop'}
best accuracy:  0.7983750000000001
